# Parse `Extremely Nested JSON` with Python: 


# **<font color=red>Mr Fugu Data Science</font>**


# (◕‿◕✿)

[Github](https://github.com/MrFuguDataScience)
[Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw?view_as=subscriber)

# Purpose & Outcome:

+ Evaluate Highly Nested Nighmare
    + Use Recursion
+ Create indexes to track nested keys
    + keep all keys (outer & inner) separated by (.) to know lineage
+ Create key:value pairs 

+ start putting back together for a dataframe

In [2]:
import pandas as pd
import collections as cc # for default dictionary

In [3]:
json_tweets=pd.read_json('tweets04.json')
json_tweets.head()
# json_tweets.iloc[10,:]

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,lang,retweeted_status,quoted_status_id,quoted_status_id_str,possibly_sensitive,extended_entities,quoted_status
0,2020-07-13 01:29:34+00:00,1282487300362956800,1282487300362956800,Gonna be wild to watch all the people engaging...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-13 01:29:17+00:00,1282487227746938881,1282487227746938880,RT @lamphieryeg: Indigenous rights. LGBTQ. Rac...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'fr', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,0,False,False,fr,{'created_at': 'Mon Jul 13 01:20:23 +0000 2020...,1.282484e+18,1.282484e+18,NaN,NaN,NaN
2,2020-07-13 01:29:05+00:00,1282487179814543360,1282487179814543360,As global economies are faced with the effects...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",NaN,NaN,...,0,False,False,en,NaN,NaN,NaN,0.0,NaN,NaN
3,2020-07-13 01:29:03+00:00,1282487169215541249,1282487169215541248,@tcoreagain @john_kilford @RobRobbEdwards the ...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.282430e+18,1.282430e+18,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-13 01:29:02+00:00,1282487164392165376,1282487164392165376,RT @NickKristof: As we scrutinize history and ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,0,False,False,en,{'created_at': 'Sat Jul 11 18:55:25 +0000 2020...,NaN,NaN,NaN,NaN,NaN


In [4]:

dct=[]
for i in json_tweets.loc[:,'retweeted_status']:
    if str(i) == 'nan':
        dct.append(['Nothing_retweeted'])
    else:
        dct.append([i])
            
# dct[:2]
# type(json_tweets.loc[:,'retweeted_status'][0])

# Iterate Through Nested Data: (`Recursively`)

+ Take `ALL keys` not just most inner or most outer!

+ Creating Index labels for all key-value pairs

+ Using Dot notation for the keys to distinguish how they are nested

+ If an entry is `NOT retweeted`: it will get labeled with a string denoting that

In [5]:
g=[]
def print_dict(v, prefix=''): 
    if isinstance(v, dict):
        for k, v2 in v.items():
            p2 = "{}.{}".format(prefix, k)
            print_dict(v2, p2)           # recursive call
    elif isinstance(v, list):
        for i, v2 in enumerate(v):
            p2 = "{}{}".format(prefix, i)
            print_dict(v2, p2)           # recursive call
    else:
#         g.append('{} : {}'.format(prefix, repr(v)))
        g.append(['{}'.format(prefix),v])
    
    return g  
print_dict(dct)

len(print_dict(dct))
# len(dct)

7026

# Separate the Index Numbers from Key:

+ If the Index doesn't have a second string, then it was `not retweeted` as you see for `['00']` 

In [7]:
def find_idx_keys_or_vals(dct):
    idx_keys=[]
    idx_vals=[]

    for j in print_dict(dct):
        idx_keys.append(j[0].split(".",1)) # split on first occurance of (.)
        idx_vals.append((j[0].split('.')[0],j[1]))
    
    return idx_keys,idx_vals


idx_keys=find_idx_keys_or_vals(dct)[0]


idx_vals=find_idx_keys_or_vals(dct)[1]

print('Index with Key:',idx_keys[:4])
print('--------------------------')
print('Index with Values:',idx_vals[:4])

Index with Key: [['00'], ['10', 'created_at'], ['10', 'id'], ['10', 'id_str']]
--------------------------
Index with Values: [('00', 'Nothing_retweeted'), ('10', 'Mon Jul 13 01:20:23 +0000 2020'), ('10', 1282484990459080705), ('10', '1282484990459080705')]


In [8]:
# Explain:
rr=[]
for i in range(len(idx_keys)-1):
    if idx_keys[i][0]==idx_keys[i+1][0]:
        rr.append((idx_keys[i][0],idx_keys[i][1:]))
rr[:5]


[('10', ['created_at']),
 ('10', ['id']),
 ('10', ['id_str']),
 ('10', ['text']),
 ('10', ['truncated'])]

In [8]:
# dd=cc.defaultdict(list)

# for i,j in idx_vals:
#         dd[i].append(j)
    
# len(dd.keys())

# dd.keys()

In [15]:
def dict_lst(idxKey_items):
    rr=[]
    for i in range(len(idxKey_items)-1):
        if idxKey_items[i][0]==idxKey_items[i+1][0]:
            rr.append((idxKey_items[i][0],idxKey_items[i][1:]))

    d=cc.defaultdict(list)

    for i,j in rr:
            d[i].extend(j)
    return d
    
idx_keys_sep=dict_lst(idx_keys)
idx_keys_sep

defaultdict(list,
            {'10': ['created_at',
              'id',
              'id_str',
              'text',
              'truncated',
              'entities.urls0.url',
              'entities.urls0.expanded_url',
              'entities.urls0.display_url',
              'entities.urls0.indices0',
              'entities.urls0.indices1',
              'metadata.iso_language_code',
              'metadata.result_type',
              'source',
              'in_reply_to_status_id',
              'in_reply_to_status_id_str',
              'in_reply_to_user_id',
              'in_reply_to_user_id_str',
              'in_reply_to_screen_name',
              'user.id',
              'user.id_str',
              'user.name',
              'user.screen_name',
              'user.location',
              'user.description',
              'user.url',
              'user.entities.url.urls0.url',
              'user.entities.url.urls0.expanded_url',
              'user.entities.url.url

In [16]:

# dd=cc.defaultdict(list)

# for i,j in idx_vals:
#         dd[i].append(j)
    
# len(dd.keys())

# dd.keys()


# Tweets_val_parse:

+ first create a dictionary where you have key:[values]

+ then iterate through the default dictionary (dd), where we append 2 lists:
    + list: (`no_tweet`) will take the index and the no tweet string
    + list2: (`tweet_vals `): takes the index again but also the values
    
+ return both list as arguments 0,1

In [17]:
def tweet_val_parse(idx_vals):
    
    dd=cc.defaultdict(list)

    for i,j in idx_vals:
        dd[i].append(j)
    
    
    no_tweet={}
    tweet_vals=cc.defaultdict(list)
    for i in dd.items():
        if 'Nothing_retweeted' in i[1]:
#             no_tweet.append((i[0],i[1][0]))
            no_tweet[i[0]]=i[1][0]
        else:
            tweet_vals[i[0]].extend(i[1])

    return no_tweet,tweet_vals


tweet_val_parse(idx_vals)[0] # index and str('Nothing_retweeted')

# tweet_val_parse(idx_vals)[1]['10'] # index and values of retweet

{'00': 'Nothing_retweeted',
 '20': 'Nothing_retweeted',
 '30': 'Nothing_retweeted',
 '70': 'Nothing_retweeted',
 '150': 'Nothing_retweeted',
 '160': 'Nothing_retweeted',
 '190': 'Nothing_retweeted',
 '200': 'Nothing_retweeted',
 '220': 'Nothing_retweeted',
 '350': 'Nothing_retweeted',
 '360': 'Nothing_retweeted',
 '420': 'Nothing_retweeted',
 '440': 'Nothing_retweeted'}

In [18]:
# c=[]

# for j in tweet_val_parse(idx_vals)[1].items():
#     for i in tweet_val_parse(idx_vals)[0].items():
#         if i[0] < j[0]:
#             c.append(i)
          
#         else:
#             c.append(j)
            
# c
# tweet_val_parse(idx_vals)[0]

# If you are considering zip(keys,values), think about it first.
 
+ Look below, the keys and values are not same length and contain hundreds of key:value pairs. You have 2 approaches:
    + 1.) Take all unique keys and then decide to zip together
    + 2.) you can do each together: but consider doing an iterator due to in memory usage otherwise.
    
+ Lastly, if you are recreating an extended Data Frame with all new columns then I would say you need to also put the non retweeted data inside of key-value pairs, or consider doing a conditional statement to fill the gaps.

In [19]:
# each retweet has different keys because, of varying factors such as links, 
# who is reposting etc. 

idx_keys_sep=dict_lst(idx_keys)


key_cnt=[]
for i in idx_keys_sep.values():
    key_cnt.append(len(i))
key_cnt[:3]


[441, 252, 222]

# Proof of Concept: Taking one entry of each and see if it can work

In [20]:
xx=[]

j=tweet_val_parse(idx_vals)[1]['10']
i=idx_keys_sep['10']

list(zip(i,j))[:5]
# tweet_val_parse(idx_vals)[1]['10']

[('created_at', 'Mon Jul 13 01:20:23 +0000 2020'),
 ('id', 1282484990459080705),
 ('id_str', '1282484990459080705'),
 ('text',
  'Indigenous rights. LGBTQ. Racism. Climate change. Orange Man Bad. https://t.co/eOouenEsqf'),
 ('truncated', False)]

# Create: Key : value pairs of nested entries

+ each entry represents a persons retweet info of the nested list

In [24]:
# keys_=idx_keys_sep.keys()
keys_=dict_lst(idx_keys).keys()

wow=[]
for ii in keys_:
    wow.append(list(zip(idx_keys_sep[ii],tweet_val_parse(idx_vals)[1][ii])))
wow[1][:3]

[('created_at', 'Sat Jul 11 18:55:25 +0000 2020'),
 ('id', 1282025722664517632),
 ('id_str', '1282025722664517632')]

In [26]:
# Grab Index and make new string(No_retweet)

notweet=[]
for i in range(len(json_tweets.loc[:,'retweeted_status'])):
    if str(json_tweets.loc[:,'retweeted_status'][i]) == 'nan':
        notweet.append((str(i)+'0','Nothing_retweeted'))
notweet

[('00', 'Nothing_retweeted'),
 ('20', 'Nothing_retweeted'),
 ('30', 'Nothing_retweeted'),
 ('70', 'Nothing_retweeted'),
 ('150', 'Nothing_retweeted'),
 ('160', 'Nothing_retweeted'),
 ('190', 'Nothing_retweeted'),
 ('200', 'Nothing_retweeted'),
 ('220', 'Nothing_retweeted'),
 ('350', 'Nothing_retweeted'),
 ('360', 'Nothing_retweeted'),
 ('420', 'Nothing_retweeted'),
 ('440', 'Nothing_retweeted')]

# Need to add the entries with correct index that were not retweeted.

+ you have 2 lists of tuples, why not append them together: no need to iterate.

+ Then we will sort by our keys

In [31]:

with_retwt=list(zip(keys_,wow))
with_retwt.extend(notweet)
# with_retwt

('40',)

# Sort by Keys for our tuples:

In [37]:
# I did not make this function got online:
def Sort_Tuple(tup):  
  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[0])  
    return tup

# Sort_Tuple(with_retwt)


# Up to you to finish this anyway you want:


In [38]:
pd.DataFrame(Sort_Tuple(with_retwt)).head()


,0,1
0,00,Nothing_retweeted
1,10,"[(created_at, Mon Jul 13 01:20:23 +0000 2020),..."
2,100,"[(created_at, Sun Jul 12 15:48:20 +0000 2020),..."
3,110,"[(created_at, Mon Jul 13 00:37:16 +0000 2020),..."
4,120,"[(created_at, Mon Jul 13 00:59:00 +0000 2020),..."


In [20]:
# dict(Sort_Tuple(with_retwt))
dct
data=dict(Sort_Tuple(with_retwt))
y=[]
keys=sorted(data.keys(), key=lambda x: int(x))

keys_=['retweeted']*len(keys)


In [38]:
import json
# json.dumps(dict(Sort_Tuple(with_retwt)),'nested_tweets.json')

# data=dict(Sort_Tuple(with_retwt))
data=list(zip(keys_,dct))
# with open("nested_tweets03.json", "w") as write_file:
#     json.dump(data, write_file)
from collections import defaultdict

data_dict = defaultdict(list)

# for i in data:
#     data_dict[i[0]].append(i[1])
# data_dict
# # with open("nested_tweets03.json", "w") as write_file:
#     json.dump(data_dict, write_file)
# json.dumps(data,sort_keys=True)

r=[]
for i in data:
    r.append({i[0]:i[1]})
with open("nested_tweets03.json", "w") as write_file:
    json.dump(r, write_file)



# <font color=red>LIKE</font> Share &

# <font color=red>SUB</font>scribe

# Citations & Help:

# ◔̯◔

https://hackersandslackers.com/extract-data-from-complex-json-python/ (this is FUNNY)

https://medium.com/@rakyll/things-i-wished-more-developers-knew-about-databases-2d0178464f78 (useful all around knowledge)

https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object (looking at what is inside of a Tweet)

https://stackoverflow.com/questions/10756427/loop-through-all-nested-dictionary-values

https://stackoverflow.com/questions/8335096/iterate-over-nested-dictionary

https://realpython.com/python-thinking-recursively/

https://www.saltycrane.com/blog/2011/10/some-more-python-recursion-examples/

https://www.geeksforgeeks.org/python-program-to-sort-a-list-of-tuples-by-second-item/